In [6]:
import pandas as pd

In [7]:
df = pd.read_csv("~/real_estate/data/v3/unsupervised_df.csv")

In [8]:
features = [
 'median_sqft_value','percent_decreasing','percent_increasing','zhvi_singlefam',
 'zhvi_1bed','zhvi_2bed','zhvi_3bed','zri_sqft_value','90_day_treasury_bill_rate',
 'mean_hourly_earnings_adj','mean_fixed_mortgage_rate','total_nonfarm_payroll',
 'people_count_lag','adult_count_lag','mean_adjusted_income_lag',
 'mean_adjusted_income_with_dep_lag','mean_real_estate_taxes_hh_lag',
 'mean_mortgage_interest_hh_lag','zhvi_condo']

states = ['AL','AR','AZ','CA','CO','CT','DC','DE','FL','GA','HI','IA','IL','IN','KY',
          'MA','ME','MI','MN','MO','NC','NE','NH','NJ','NV','NY','OH','OK','OR','PA',
          'RI','SC','TN','TX','VA','WA','WI','WV']

target = "zhvi_condo"

In [9]:
# Store data from previous, future months and years
feature_mo = {}
feature_yr = {}
zhvi_condo_12_months = []

for i in features:
    feature_mo[i+"_mo"] = []
    feature_yr[i+"_yr"] = []

for i in df.index:
    
    current_row = df.iloc[i]
    region_name = current_row["RegionName"]
    month = int(current_row["month"])
    year = int(current_row["year"])
    next_year = year + 1
    
    # Get column values of previous time preiods
    last_year = year - 1
    if month == 1:
        last_month_month = 12
        last_month_year = year - 1
    else:
        last_month_month = month - 1
        last_month_year = year
    
    last_month_row = df.loc[(df["RegionName"] == region_name) & 
                            (df["month"] == last_month_month) &
                            (df["year"] == last_month_year)]
    
    last_year_row = df.loc[(df["RegionName"] == region_name) &
                           (df["month"] == month) &
                           (df["year"] == last_year)]
    
    next_year_row = df.loc[(df["RegionName"] == region_name) &
                           (df["month"] == month) &
                           (df["year"] == next_year)]
    
    for i in features:
        
        feature_mo[i+"_mo"].append(last_month_row[i].values)
        feature_yr[i+"_yr"].append(last_year_row[i].values)
        
    next_year_target = next_year_row["zhvi_condo"]
    
    if len(next_year_target) == 0:
        next_year_target = None
    else:
        next_year_target = next_year_target.values[0]
    
    zhvi_condo_12_months.append(next_year_target)
    
# Add data to dataframe
for i in feature_mo.keys():
    new_list = [j[0] if len(j) > 0 else None for j in feature_mo[i]]
    feature_mo[i] = new_list

for i in feature_yr.keys():
    new_list = [j[0] if len(j) > 0 else None for j in feature_yr[i]]
    feature_yr[i] = new_list
    
for i in feature_mo.keys():
    df[i] = feature_mo[i]
    
for i in feature_yr.keys():
    df[i] = feature_yr[i]
    
df["zhvi_condo_12_months"] = zhvi_condo_12_months

In [10]:
# Filter
df = df.dropna()

# Export
df.to_csv("~/real_estate/data/v3/supervised_df.csv", index=False)

In [11]:
print(df.shape)
df.head()

(82733, 99)


,RegionName,median_sqft_value,percent_decreasing,percent_increasing,zhvi_1bed,zhvi_2bed,zhvi_3bed,zri_sqft_value,zhvi_condo,zhvi_singlefam,...,mean_adjusted_income_with_dep_lag_yr,mean_adjusted_income_lag_yr,percent_increasing_yr,mean_mortgage_interest_hh_lag_yr,total_nonfarm_payroll_yr,zhvi_2bed_yr,zhvi_3bed_yr,adult_count_lag_yr,mean_fixed_mortgage_rate_yr,zhvi_condo_12_months
11778,94109,682.0,54.79,35.10,481300.0,746200.0,1190200.0,3.196,674600.0,2242600.0,...,71.772723,78.140829,28.87,2.579651,130761.0,768100.0,1178000.0,35604.0,4.3,801700.0
11779,20002,308.0,28.67,54.74,290500.0,346000.0,368400.0,1.824,296600.0,376900.0,...,37.897626,48.172472,44.82,4.216171,130761.0,343500.0,363700.0,28526.0,4.3,322100.0
11780,90046,479.0,57.95,30.91,307300.0,634300.0,930600.0,2.298,405700.0,945700.0,...,56.279705,61.899775,20.34,4.565220,130761.0,652300.0,1096300.0,31344.0,4.3,422400.0
11781,20009,503.0,40.72,37.54,331900.0,512700.0,681600.0,3.048,388900.0,714100.0,...,59.339971,65.897121,48.06,4.579302,130761.0,508900.0,684100.0,31710.0,4.3,406700.0
11782,7030,463.0,69.90,17.66,330500.0,477500.0,901300.0,2.656,446100.0,951000.0,...,88.343339,99.600399,18.35,5.010871,130761.0,485200.0,889700.0,33055.0,4.3,474700.0
